In [9]:
import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..','fly_model'))
sys.path.insert(0, parent_dir)
from Model import Model
import matplotlib.pyplot as plt
path_to_frame = 'G:/My Drive/Research/gaussian_splatting/gaussian_splatting_input/mov19_2022_03_03/'
path_to_mesh = 'G:/My Drive/Research/gaussian_splatting/mesh'

model = Model(path_to_mesh, path_to_frame)

joint_to_update = [model.right_wing_skeleton_root,model.left_wing_skeleton_root]

rotation = [[30,0,0],[-30,0,0]]
[model.update_local_rotation(joint_to_update,rotation) for joint_to_update,rotation in zip(joint_to_update,rotation)]
global_rotated,global_normals = model.update_skin_and_joints()


d:\Documents\gaussian_splat\model\fly_model\Skin.py:21: RuntimeWarning:

invalid value encountered in divide



In [2]:
import Plotter
import plotly.graph_objects as go

fig = go.Figure()
Plotter.plot_skeleton(model.all_skin[0].bones,fig,marker_dict = {'size': 3},line_dict ={'width': 3})
Plotter.plot_skin(fig,global_rotated,model.fly_skin,'body',color = model.fly_skin.weights[::10,3],skip_skin_points = 10)
# Plotter.plot_skin(fig,all_skin,left_wing,'left wing')

fig.show()

In [10]:
global_rotated.shape

(123580, 3)

In [1]:

# load model

import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..','fly_model'))
sys.path.insert(0, parent_dir)
import numpy as np
from Skin import Skin
import plotly.graph_objects as go
from Joint import Joint
from Frame import Frame
from Camera import Camera
import scipy

import numpy as np
import Plotter
from Skeleton import Skeleton
from Skin import Skin
import plotly.graph_objects as go
import Utils
import pandas as pd
%matplotlib qt

path_to_mesh = 'I:/My Drive/Research/gaussian_splatting/mesh'
skin_parts = ['body','right_wing','left_wing']
scale = 1/1000
skin_translation = np.array([-0.1-1,0,1])*scale

# initilize skeletone, joints and bones
# body angles - yaw,pitch,roll
# wing angles - phi, psi, theta
pitch_body = 0


root_no_bone = Joint([1,0,0],[0,-pitch_body,0],parent = None, end_joint_of_bone = False, scale = scale)
neck = Joint([0.6,0,0.3],[0,pitch_body,0],parent = root_no_bone, end_joint_of_bone = False, scale = scale)
neck_thorax =  Joint([0.6,0,0.3],[0,-25,0], parent = root_no_bone, end_joint_of_bone = False, scale = scale)
head  =Joint([0.3,0,0],[0,0,0], parent = neck, scale = scale)
thorax  =Joint([-1,0,0],[0,25,0], parent= neck_thorax ,scale = scale)
abdomen = Joint([-1.3,0,0],[0,0,0], parent = thorax, scale = scale)
right_sp_no_bone = Joint([0,0,0.3],[0,pitch_body,0],parent = root_no_bone, end_joint_of_bone = False , scale = scale, color = 'red', rotation_order = 'zxy')
right_wing_root = Joint([0,-0.3,0],[0,0,0], parent = right_sp_no_bone, end_joint_of_bone = False, scale = scale, color = 'red',rotation_order = 'zxy')
right_wing_tip = Joint([0,-2.2,0],[0,0,0], parent = right_wing_root, scale = scale, color = 'red',rotation_order = 'zxy')
left_sp_no_bone = Joint([0,0,0.3],[0,pitch_body,0], parent = root_no_bone, end_joint_of_bone = False, scale = scale, color = 'blue',rotation_order = 'zxy')
left_wing_root = Joint([0,0.3,0],[0,0,0],parent = left_sp_no_bone, end_joint_of_bone = False, scale = scale, color = 'blue',rotation_order = 'zxy')
left_wing_tip = Joint([0,2.2,0],[0,0,0], parent =left_wing_root, scale = scale, color = 'blue',rotation_order = 'zxy')


body = Skin(f'{path_to_mesh}/body.stl',scale = 1.05,color = 'lime')
right_wing = Skin(f'{path_to_mesh}/right_wing.stl',scale = 1.05,constant_weight = right_wing_root,color = 'crimson')
left_wing = Skin(f'{path_to_mesh}/left_wing.stl',scale = 1.05, constant_weight = left_wing_root,color = 'dodgerblue')


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


c:\Users\Roni\Documents\gaussian_splat\gaussian_splat\model\fly_model\Skin.py:21: RuntimeWarning:

invalid value encountered in divide



In [2]:
# initilize joints
joints_of_bone = root_no_bone.get_and_assign_bones()
[skin.add_bones(joints_of_bone) for skin in  [body, right_wing,left_wing]]
[skin.translate_ptcloud_skin(skin_translation) for skin in  [body, right_wing,left_wing]]
body.calculate_weights_dist(body.bones[0:3])
right_wing.calculate_weights_constant()
left_wing.calculate_weights_constant()

joint_list = root_no_bone.get_list_of_joints()
list_joints_pitch_update = [neck,right_sp_no_bone,left_sp_no_bone]


# all_skin_points = [skin.rotate_skin_points() for skin in [body,right_wing,left_wing]]
# all_skin_normals = [skin.rotate_skin_normals() for skin in [body,right_wing,left_wing]]




In [3]:
# load frames and cameras
from Frame import Frame

path = 'I:/My Drive/Research/gaussian_splatting/gaussian_splatting_input/mov19_2022_03_03/'
frames = list(range(900,970,1))
image_name= []
for frame in frames:
    image_name += [f'P{frame}CAM{cam + 1}' for cam in range(4)]

frames = {f'{im_name}.jpg':Frame(path,im_name,idx) for idx,im_name in enumerate(image_name)}


In [5]:
# hull reconstruction 

def get_hull_part(part,real_coord):
    points_3d_frame = points_3d[part][points_3d[part]['frame'] == 900]
    real_coord_frame = real_coord[real_coord[:,3] == 900,:]
    points_in_ew_frame = np.array([real_coord_frame[points_3d_frame[ax] - 1,idx] for idx,ax in enumerate(['X','Y','Z'])]).T
    points_in_ew_frame  = np.column_stack((points_in_ew_frame,np.arange(1,points_in_ew_frame.shape[0] + 1)))
    return points_in_ew_frame
points_3d = {body_wing : pd.DataFrame(Utils.load_hull(body_wing,path),columns = ['X','Y','Z','frame']) for body_wing in ['body','rwing','lwing']}
real_coord = scipy.io.loadmat(f'{path}/3d_pts/real_coord.mat')['all_coords']

hull = [get_hull_part(part,real_coord) for part in list(points_3d.keys())]

In [4]:
frame_number = 910 
frame_names = ['P900CAM1.jpg','P900CAM2.jpg','P900CAM3.jpg','P900CAM4.jpg']
frame_names = [f'P{frame_number}CAM{idx}.jpg' for idx in range(1,5)]

camera_pixel = np.vstack([frames[frame].camera_center_to_pixel_ray(frames[frame].cm) for frame in  frame_names])
camera_center = np.vstack([frames[frame].X0.T for frame in  frame_names])
rot_mat_ew_to_lab = frames['P900CAM1.jpg'].rotation_matrix_from_vectors(frames['P900CAM1.jpg'].R[2,:], [0,0,1])
cm_point = Utils.triangulate_least_square(camera_center,camera_pixel)
cm_point_lab = np.squeeze(np.dot(rot_mat_ew_to_lab,cm_point[:,np.newaxis]).T)




In [6]:

fig = go.Figure()
Plotter.plot_skeleton(body.bones,fig,marker_dict = {'size': 3},line_dict ={'width': 3})
Plotter.plot_skin(fig,body.ptcloud_skin,body,'body',skip_skin_points = 1, size = 1)
Plotter.plot_skin(fig,right_wing.ptcloud_skin,right_wing,'right wing',size = 1,skip_skin_points = 1)
Plotter.plot_skin(fig,left_wing.ptcloud_skin,left_wing,'left wing',size = 1,skip_skin_points = 1)
# Plotter.plot_skin(fig,all_skin,left_wing,'left wing')
4
fig.show()

In [11]:
cm_point_lab + np.array([-0.2,-0.4,0.22])*1/1000

array([-0.00154725,  0.00540915,  0.00833845])

In [8]:
# update model position

pitch = -47
root_no_bone.set_local_translation([-0.0015,0.0053,0.0083])
root_no_bone.set_local_rotation([244,pitch,-5])
[joint.set_local_rotation([0,-pitch,0]) for joint in list_joints_pitch_update]
# thorax.set_local_rotation([0,-10,0])
right_wing_root.set_local_rotation([-17,-119,6])
left_wing_root.set_local_rotation([33,-115,-6])
[joint.update_rotation() for joint in joint_list]

all_skin_points_parts = [skin.rotate_skin_points() for skin in [body,right_wing,left_wing]]
all_skin_normals_parts = [skin.rotate_skin_normals() for skin in [body,right_wing,left_wing]]
all_skin = np.unique(np.vstack(all_skin_points_parts),axis = 0)

In [10]:
import matplotlib.pyplot as plt
import pickle
name_folder = f'model_pose_time_color_v2'
key = 969
model_path = os.path.join(f"I:/My Drive/Research/gaussian_splatting/gaussian_splatting_output/gaussian_splattimg_output_model/{name_folder}/", f'{key}/body_wing_pose.pkl')

with open(model_path, 'rb') as file:
    data_dict_original = pickle.load(file)

plt.figure(),plt.plot(data_dict_original['right_wing'][:,0])
plt.plot(data_dict_original['left_wing'][:,0])
plt.figure(),plt.plot(data_dict_original['right_wing'][:,1])
plt.plot(data_dict_original['left_wing'][:,1])
plt.figure(),plt.plot(data_dict_original['right_wing'][:,2])
plt.plot(data_dict_original['left_wing'][:,2])

for k in range(3):
    plt.figure(),plt.plot(data_dict_original['body_angle'][:,k])
    plt.plot(data_dict_original['body_angle'][:,k])

In [12]:
# update model position

pitch = -47

idx = 0
root_no_bone.set_local_translation(data_dict_original['body_location'][idx])
root_no_bone.set_local_rotation(data_dict_original['body_angle'][idx])
[joint.set_local_rotation([0,-pitch,0]) for joint in list_joints_pitch_update]
# thorax.set_local_rotation([0,-10,0])
right_wing_root.set_local_rotation(data_dict_original['right_wing'][idx])
left_wing_root.set_local_rotation(data_dict_original['left_wing'][idx])
[joint.update_rotation() for joint in joint_list]

all_skin_points_parts = [skin.rotate_skin_points() for skin in [body,right_wing,left_wing]]
all_skin_normals_parts = [skin.rotate_skin_normals() for skin in [body,right_wing,left_wing]]
all_skin = np.unique(np.vstack(all_skin_points_parts),axis = 0)



In [17]:

fig = go.Figure()
Plotter.plot_skeleton(body.bones,fig,marker_dict = {'size': 3},line_dict ={'width': 3})
Plotter.plot_skin(fig,all_skin_points_parts[0],body,'body',skip_skin_points = 1)
Plotter.plot_skin(fig,all_skin_points_parts[1],right_wing,'right wing',skip_skin_points = 1)
Plotter.plot_skin(fig,all_skin_points_parts[2],left_wing,'left wing',skip_skin_points = 1)
# Plotter.plot_skin(fig,all_skin,left_wing,'left wing')

fig.show()

In [93]:
[-0.0015,0.0054,0.0083]
[249,-25,0]
[-27,-115,10]

array([[-0.00097193,  0.00682449,  0.00753745],
       [-0.00097616,  0.00682616,  0.00753807],
       [-0.0010249 ,  0.00684294,  0.00753453],
       ...,
       [ 0.000551  ,  0.00596178,  0.00839313],
       [ 0.0004502 ,  0.00595886,  0.00841551],
       [ 0.00055223,  0.00595596,  0.0084    ]])

In [13]:
# rotate to easy wand and project to 2d
import matplotlib.pyplot as plt
frame_number = 900

frame_name = f'P{frame_number}CAM4.jpg'
all_skin_in_ew = np.dot(rot_mat_ew_to_lab.T,all_skin.T).T
projeced = frames[frame_name].project_with_proj_mat(all_skin_in_ew)
fig,ax = plt.subplots(2,2)

for idx in range(4):
    all_skin_in_ew = np.dot(rot_mat_ew_to_lab.T,all_skin.T).T
    projeced = frames[f'P{frame_number}CAM{idx + 1}.jpg'].project_with_proj_mat(all_skin_in_ew)
    ax[idx//2,np.mod(idx,2)].imshow(frames[f'P{frame_number}CAM{idx + 1}.jpg'].image_with_bg, cmap = 'gray')
    ax[idx//2,np.mod(idx,2)].scatter(projeced[:,0],projeced[:,1],color = 'red', alpha = 0.05, s= 2,cmap = 'gray')


C:\Users\Roni\AppData\Local\Temp\ipykernel_18496\3991537193.py:14: UserWarning:

No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored



In [ ]:
# generate frames dictionary



frames_dict = {}
for frame_number in range(900,970):
    


    cams,base_images = {},{}
    for idx in range(4):
        frame_name = f'P{frame_number}CAM{idx+1}.jpg'
        frame = frames[frame_name]
        all_skin_in_ew_homo = frame.homogenize_coordinate(all_skin_in_ew)
        frame.map_3d_2d(all_skin_in_ew_homo, croped_image = False,use_zbuff = True)
        base_images[frame.image_id] = frame.generate_base_image()
        frame.save_croped_images()
        cams[frame.camera_number] = frame.cams_for_gs()
    frames_dict[frame_number] = [base_images,cams,[frame.voxels_with_idx[:,0:3],np.full(frame.voxels_with_idx[:,0:3].shape,int(100))]]

Utils.pickle_file(frames_dict,f'{frame.path}/input_data_for_gs/sparse/frames_model.pkl' )


In [ ]:
data_dict_original['']

In [80]:
data_dict_original['left_wing'][0]

array([  34.941555, -116.3183  ,   -6.587137], dtype=float32)

In [61]:
plt.figure(),plt.plot(data_dict_original['right_wing'][:,1],data_dict_original['right_wing'][:,0])

(<Figure size 640x480 with 1 Axes>,
 [<matplotlib.lines.Line2D at 0x14b20fb6020>])

In [53]:
data_dict_original['right_wing'][:,1]

array([-120.39326 , -128.65698 , -131.7047  , -131.86008 , -130.45741 ,
       -129.21692 , -126.21735 , -125.73161 , -121.94799 , -119.584785,
       -113.76786 , -109.58079 , -103.78034 , -100.68551 ,  -93.3805  ,
        -91.32281 ,  -86.05099 ,  -85.56294 ,  -86.92758 ,  -90.72246 ,
       -101.35927 , -108.77717 , -115.70123 , -116.01024 , -113.11601 ,
       -103.35256 , -100.02493 ,  -98.59268 ,  -93.4529  ,  -92.16294 ,
        -88.626945,  -85.65033 ,  -74.43937 ,  -65.00436 ,  -55.864788,
        -51.943813,  -47.335247,  -43.23212 ,  -40.85217 ,  -35.941032,
        -33.54621 ,  -32.977886,  -35.353325,  -39.722183,  -41.249855,
        -43.374043,  -44.960377,  -46.315186,  -50.139603,  -54.086468,
        -58.12327 ,  -61.566113,  -65.1902  ,  -70.096085,  -73.96886 ,
        -80.13898 ,  -88.33187 ,  -92.247665,  -97.011856,  -99.53819 ,
       -102.21485 , -106.91495 , -106.91686 , -110.81933 , -113.009056,
       -118.42314 , -120.86291 , -132.27844 , -153.85031 , -165.

In [ ]:
# load frames and cameras

path = 'G:/My Drive/Research/gaussian_splatting/gaussian_splatting_input/mov19_2022_03_03/'
frames = list(range(900,910,1))
image_name= []
for frame in frames:
    image_name += [f'P{frame}CAM{cam + 1}' for cam in range(4)]

frames = {f'{im_name}.jpg':Frame(path,im_name,idx) for idx,im_name in enumerate(image_name)}


In [ ]:
path = 'G:/My Drive/Research/gaussian_splatting/gaussian_splatting_input/mov19_2022_03_03/'
# map 3d voxels to 2d pixels    

[frames[im_name].map_3d_2d(croped_image = True, use_zbuff = True) for im_name in frames.keys()]
voxel_dict,colors_dict = Utils.get_dict_for_points3d(frames)
ggs = GenerateGsInput(path,frames)
frames_dict = {}
idx = 0
for frame in range(1407,1430):
    
    image_name,points_in_idx = Utils.define_frames([frame],points_3d)
    cameras = {f'cam{cam + 1}':Camera(path,cam) for cam in range(4)}
    frames = {f'{im_name}.jpg':Frame(path,im_name,points_in_idx[im_name.split('CAM')[0]],real_coord,idx) for idx,im_name in enumerate(image_name)}
    [frames[im_name].map_3d_2d(croped_image = True, use_zbuff = True) for im_name in frames.keys()]
    voxel_dict,colors_dict = Utils.get_dict_for_points3d(frames)
    ggs = GenerateGsInput(path,frames)
    
    ggs.save_croped_images(croped_image = True)
    ggs.generate_camera_text(croped_image = True)
    ggs.generate_image_text()
    ggs.generate_points3d_text(voxel_dict,colors_dict)
    base_images = ggs.generate_base_image()
    cams = ggs.generate_cams(croped_image = True)
    xyz,rgb = ggs.generate_xyz_rgb(voxel_dict,colors_dict)
    frames_dict[frame] = [base_images,cams,[xyz,rgb]]
    idx += 1


Utils.pickle_file(frames_dict,f'{ggs.sparse_dir}/frames_zbuff.pkl' )


In [12]:
frames[frame_name].map_3d_2d(all_skin_in_ew_homo, croped_image = False,use_zbuff = True)
frames[frame_name].color_of_pixel

array([  3,   5,  11, ..., 124, 124, 124], dtype=uint16)

In [8]:
# get_color from image and plot
all_skin_in_ew_homo = frames[frame_name].homogenize_coordinate(all_skin_in_ew)
frames[frame_name].map_3d_2d(all_skin_in_ew_homo, croped_image = False,use_zbuff = True)
voxel_dict,colors_dict = Utils.get_dict_for_points3d(frames)


AttributeError: 'Frame' object has no attribute 'color_of_pixel'

In [13]:
fig = go.Figure()
Plotter.plot_skin(fig,all_skin[::,:],right_wing,'body', color = frames[frame_name].color_of_pixel[::10],colorscale = 'gray')
fig.show()

AttributeError: module 'Utils' has no attribute 'get_dict_for_points3d'

In [100]:
scale_camera_center = np.vstack((np.array([[0.26,0.16,0.16,0.16]]),np.array([[0.3,0.22,0.22,0.22]]))).T
scale_vectors = np.vstack((np.array([[0.26,0.16,0.16,0.16]]),np.array([[0.3,0.22,0.22,0.22]]))).T


rotated_vec = [Utils.rotate_vector_direction_and_clip(rot_mat_ew_to_lab, np.vstack((center,pixel)), scale[:,np.newaxis]) for center,pixel,scale in  zip(camera_center,camera_pixel,scale_vectors)]

points_in_ew_frame_rotated = [np.dot(rot_mat_ew_to_lab,part[:,0:3].T).T for part in hull]


In [101]:

fig = go.Figure()
Plotter.scatter3d(fig,all_skin[::10,:],'skin', mode = 'markers',marker_dict =  {'color': 'black'}) 

Plotter.scatter3d(fig,points_in_ew_frame_rotated[0],'hull_body', mode = 'markers',marker_dict =  {'color': 'blue'}) 
Plotter.scatter3d(fig,points_in_ew_frame_rotated[1],'hull_rwing', mode = 'markers',marker_dict = { 'color': 'magenta'}) 
Plotter.scatter3d(fig,points_in_ew_frame_rotated[2],'hull_lwing', mode = 'markers',marker_dict = {'color':'cyan'}) 

marker_dict_point = {'size': 5, 'color': 'magenta'}

Plotter.scatter3d(fig,cm_point_lab[np.newaxis],'least square', mode = 'markers',marker_dict = {'size': 5, 'color': 'blue'}) 

[Plotter.scatter3d(fig,rotated_vec,idx, mode = 'lines+markers',marker_dict = {'size': 5, 'color': [[255,0,0],[0,255,0]]}) for rotated_vec,idx in zip(rotated_vec,range(4))]
# [Plotter.scatter3d(fig,np.vstack((origin,end)),idx, mode = 'lines',line_dict = {'width': 3, 'color': 'red'}) for origin,end,idx in zip(camera_center_lab, camera_pixel_lab,range(4))]

fig.show()